In [8]:
from pymatgen.core import Structure
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [21]:
[x for x in np.unique(py_strucs[0].species) if x.is_metal]

[Element Fe]

In [22]:
df = pd.read_csv("../data/data.csv")

In [25]:
py_strucs = [Structure.from_file("../data/cifs/rsm/%s.cif" %c) for c in tqdm(df["cif.label"])]

  0%|          | 3/3257 [00:00<02:18, 23.58it/s]/Users/xiaoqi/opt/miniconda3/envs/mofgraph2vec/lib/python3.8/site-packages/pymatgen/io/cif.py:1147: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
100%|██████████| 3257/3257 [01:37<00:00, 33.54it/s]


In [30]:
metal_list = [[x.symbol for x in np.unique(struc.species) if x.is_metal] for struc in tqdm(py_strucs)]

100%|██████████| 3257/3257 [00:08<00:00, 365.11it/s]


In [31]:
for x in metal_list:
    if len(x) != 1:
        print(x)

['Gd', 'Ag']
['Cu', 'W']
['Al', 'Ag']
['Gd', 'Co']
['Cd', 'Ni']
['Na', 'Cd']
['Cu', 'Mo']
['K', 'La']
['K', 'Ce']
['Na', 'Zn']
['K', 'Zn']
['Cd', 'Ni']
['Cu', 'W']
['Tb', 'Cu']
['Mn', 'Fe']
['K', 'Sc']
['Cu', 'Mo']
['Cu', 'W']
['Zn', 'Co']
['Ca', 'Pb']
['Mn', 'Cu']
['Li', 'Zn']
['Lu', 'W']
['Na', 'Zn']
['Rb', 'Co']
['Ba', 'Cd']
['Na', 'Co']
['Zn', 'Cu']
['Zn', 'Co']
['K', 'Ni']
['Fe', 'Ni']
['Cu', 'Mo']
['Cu', 'Mo']
['La', 'Co']
['Ni', 'W']
['Ni', 'Mo']
['La', 'Cu']
['Ca', 'Cd']
['Ni', 'Mo']
['Ho', 'Cu']
['Na', 'Tb']
['La', 'Mg']
['Mn', 'Cu']
['Mn', 'Co']
['Eu', 'W']
['Ba', 'Cu']
['Gd', 'Ag']
['Cu', 'W']
['Mn', 'Cr', 'Fe']
['Cu', 'W']
['Sr', 'Zn']
['Sr', 'Ni']
['La', 'Eu']
['Ca', 'Ce']
['Ba', 'Zn']
['Ba', 'Co']
['Cu', 'W']
['Tb', 'Eu']
['Ni', 'Mo']
['Tb', 'Cu']
['Tb', 'Cu']
['Sm', 'Zn']
['Na', 'Zn', 'Co']
['Eu', 'Ag']
['Ce', 'Mg']
['Nb', 'Cu']
['Na', 'Zn']
['Na', 'Zn', 'Fe']
['Mg', 'Cd', 'Mo']
['Mn', 'Au']
['Mn', 'Ag']
['La', 'Ni']
['K', 'Fe', 'Ni']
['Co', 'Cu']
['Sr', 'U']
['Na', 'Co'

In [32]:
df["metal_1"] = [ele[0] for ele in metal_list]

In [36]:
metal_2 = [ele[1] if len(ele)>1 else np.NaN for ele in metal_list]

In [44]:
metal_3 = [ele[2] if len(ele)>2 else np.NaN for ele in metal_list]

In [45]:
metal_4 = [ele[3] if len(ele)>3 else np.NaN for ele in metal_list]

In [43]:
df["metal_2"] = metal_2

In [46]:
df["metal_3"] = metal_3

In [47]:
df["metal_4"] = metal_4

In [49]:
df.loc[:, ~df.columns.str.match('Unnamed: 0')]

,in,ASA [m^2/cm^3],CellV [A^3],Df,Di,Dif,NASA [m^2/cm^3],POAV [cm^3/g],POAVF,PONAV [cm^3/g],...,KH_N2_molkgpa,log_selectivity,info.decorated_graph_hash,info.undecorated_graph_hash,info.decorated_scaffold_hash,info.undecorated_scaffold_hash,metal_1,metal_2,metal_3,metal_4
0,8,2163.580,20019.00,7.83744,9.47418,9.45292,0.000000,1.074720,0.64186,0.000000,...,4.160970e-06,0.449093,382517780dc5ab8eb8fa43295ce8bdf4,b49059acbd8c344ef658b5b181a142f5,25800f339ff3d43d2ddc5d1a59b4eeb4,08515bc3d51c5d80eafef0baa6e2b134,Cu,NaN,NaN,NaN
1,9,641.266,2633.04,4.14331,4.82084,4.80387,0.000000,0.136436,0.24024,0.000000,...,7.416090e-08,NaN,a84c197bbfa5810808c44dc4ccfe11af,874ad4e1f85e1de8bd54bed65199e13a,b526ee2136f2d05af53b2bed85c5339d,b02ed13751c7be6d0f820f3e5f81b9b3,La,NaN,NaN,NaN
2,10,1603.800,15518.00,13.71867,18.80790,18.79960,0.888134,0.957188,0.64550,0.000000,...,6.027810e-06,0.533550,45efa7c1e878af5ecbc618b706fc1136,bccd7095b8e6d26f8257ae674095d645,13cae2b9e4cfe2cfe9d05a9468ae4830,eb2900477384511abd851c626625231e,In,NaN,NaN,NaN
3,11,600.107,2602.59,4.06893,4.80529,4.80529,0.000000,0.133609,0.23884,0.000000,...,1.174370e-06,0.522879,67abee315179d34012433698bf59a924,9fa41a3543983b720923f5ffb080b005,57a7b96939ec88db393126c63b0ed48d,0a8666fb6118b8149daf8d5e89321c93,Ce,NaN,NaN,NaN
4,15,498.734,2493.02,3.94356,4.70447,4.68752,0.000000,0.113502,0.21898,0.000000,...,1.454260e-06,0.548749,3f69a0844ce43a2c27086353118d05ed,44a150af84481dfe02a60f89af03568c,8cd5110f581b241624b3544555208c56,9be47b63130f6c51d0ab9c1bf63e789e,Eu,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,9677,601.756,4479.64,4.08689,4.75984,4.75984,0.000000,0.163833,0.25402,0.000000,...,3.365250e-06,0.868569,df97f8d55b270ccf87d6b5ef1d240b49,b9fbb38978dc92454d5f9dadc2d1f889,b7ce948b3839045aeca55c2b2b8c5831,04093fb5811c815186f3c99d9bf510d0,Cd,NaN,NaN,NaN
3253,9679,1607.600,2134.77,7.69493,9.54888,9.54888,0.000000,0.661485,0.57334,0.000000,...,3.589710e-06,0.509364,bcd37c930e2b58a516766833377a8c77,d7f19b4984eefc1dfe81e7e5d11a2c3c,3363bfcc7baaa3e433d11644fc5fa6ee,4cd3ef34cda4fc1a43dccccec19778b1,Zn,NaN,NaN,NaN
3254,9681,416.131,1636.02,4.20115,4.73990,4.72096,0.000000,0.093092,0.16366,0.000000,...,7.976930e-06,1.153723,f5a545a2d506d8424bfbbfd2917b9b9b,006d95bcc7ab668a21894bcb66e6b445,5a0a0630fef8a2bafe333d8fe4388354,ba67a8fe5b11dba57bc563a5d3b7a543,Zn,NaN,NaN,NaN
3255,9685,0.000,2044.51,2.56035,4.18124,4.16282,35.643400,0.000000,0.00000,0.024759,...,3.393710e-06,NaN,47335f2329fe60935ca636a63a0ebd48,70e36dc2a0dd36e20a761769501aa3f9,f0900a12ad6d994e6c6a490cedc50406,55501fe2ad68169f472605c0ab6f6cd2,K,Gd,NaN,NaN


In [50]:
df.to_csv("../data/rsm_metal.csv", index=False)